In [148]:
cwd = os.path.abspath(r'C:\Users\oazed\AlphaTrade Dropbox\Alpha Trade Pty Limited\Operation\Statements\GCEX\Daily Statements')
files = os.listdir(cwd)


#### ALL the CFDS , CRYPTOs, and FX Vals

strathos_714=[]
lis3= []

excel_files = [x for x in os.listdir(cwd) if x.endswith(".xls")]

  
    
for file in excel_files:
    if file.startswith('714'):
#         if '2021-12-14' in file:  ## BREAK IT OR UNTIL U WANT
#             break
        lis3.append(file)
                    
for i in range(lis3.index("714_2021-12-01_2021-12-01.xls"),len(lis3)):  ## CHANGE THE DATE & MONTH HERE
    excel_path =os.path.join(cwd,lis3[i])
    this_df = pd.read_excel(excel_path)
    this_df = this_df.rename(columns={'GC Exchange Limited': 'col1'})
    this_df['filename'] = lis3[i]
    try:
        this_df = this_df.iloc[this_df.col1[this_df.col1 == 'Executed Exchange Trades'].index.tolist()[0]+1:this_df.col1[this_df.col1 == 'Closed Exchange Trades'].index.tolist()[0]+1]
    except IndexError:
        this_df = this_df.iloc[this_df.col1[this_df.col1 == 'Executed CFD Trades'].index.tolist()[0]+1:this_df.col1[this_df.col1 == 'Closed CFD Trades'].index.tolist()[0]+1]
    strathos_714.append(this_df)
    strath = pd.concat(strathos_714)
    


## FOR FX


strathos_fx=[]
lis_stra=[] 
    
    

for file in excel_files:
    if file.startswith('714'):
#         if '2021-12-14' in file:  ## BREAK IT OR UNTIL U WANT
#             break
        lis_stra.append(file)             
for i in range(lis_stra.index("714_2021-12-01_2021-12-01.xls"),len(lis_stra)):  ## CHANGE THE DATE & MONTH HERE
    excel_path =os.path.join(cwd,lis_stra[i])
    this_df = pd.read_excel(excel_path)
    this_df = this_df.rename(columns={'GC Exchange Limited': 'col1'})
    this_df['filename'] = lis_stra[i]
    try:
        this_df = this_df.iloc[this_df.col1[this_df.col1 == 'Executed Trades'].index.tolist()[0]+1:this_df.col1[this_df.col1 == 'Executed Exchange Trades'].index.tolist()[0]+1]
    except IndexError:
        this_df = this_df.iloc[this_df.col1[this_df.col1 == 'Executed Trades'].index.tolist()[0]+1:this_df.col1[this_df.col1 == 'Executed CFD Trades'].index.tolist()[0]+1]
    strathos_fx.append(this_df)
    stra = pd.concat(strathos_fx)
    


trades = pd.concat([strath])
trades = trades.drop(['Unnamed: 10'], axis=1)
trades = trades.dropna().reset_index()
df_tr = trades[trades['col1'].apply(lambda x: "Trade Date" in x)]
trades = trades.drop(df_tr.index)
trades = trades.drop(['index'], axis=1)
trades = trades[['col1', 'Unnamed: 3','Unnamed: 4','Unnamed: 7','Unnamed: 8','Unnamed: 9','filename' ]]
trades = trades.rename(columns={'col1': 'Date', 'Unnamed: 3':'Execution ID', 'Unnamed: 4':'Instrument', 'Unnamed: 7':'Rate', 'Unnamed: 8':'Dealt Qty', 'Unnamed: 9':'Settle CCY', 'filename':'AccName'})
trades.AccName = trades.AccName.str[0:3]
trades['Rate'].replace(to_replace=",", value="", regex=True, inplace=True)
trades['Dealt Qty'].replace(to_replace=",", value="", regex=True, inplace=True)
trades[["Rate", "Dealt Qty"]] = trades[["Rate", "Dealt Qty"]].apply(pd.to_numeric)
trades.rename(columns={'Date':'date','Instrument':'instrument'}, inplace=True)
trades["date"] = pd.to_datetime(trades["date"])
tr_purge = trades[trades['Execution ID'].apply(lambda x: len(x)>19)]
trades = trades.drop(tr_purge.index).reset_index()
trades= trades.drop(['Execution ID'],axis=1)
def banao(x):
    bana = x['instrument']
    x = bana.split('/')
    return x[1]+'/'+x[2]
settle = trades.apply(banao,axis=1)
trades['instrument'] = settle
trades= trades.drop(['index'],axis=1)

fx_trades = pd.concat([stra]) 
fx_trades = fx_trades.dropna()
fx_trades= fx_trades.reset_index()
fx_trades= fx_trades.drop(['index'], axis=1)
df_c = fx_trades[fx_trades['col1'].apply(lambda x: "Trade Date" in x)]
fx_trades = fx_trades.drop(df_c.index)
fx_trades = fx_trades[['col1', 'Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 10','filename' ]]
fx_trades = fx_trades.rename(columns={'col1': 'Date', 'Unnamed: 3':'ID','Unnamed: 4':'Instrument', 'Unnamed: 5':'Side', 'Unnamed: 6':'Dealt Amount', 'Unnamed: 7':'CCY One', 'Unnamed: 8':'Rate', 'Unnamed: 9':'Contra Qty', 'Unnamed: 10':'CCY Two', 'filename':'AccName'})
fx_trades= fx_trades.reset_index()
fx_trades= fx_trades.drop(['index'], axis=1)
fx_trades.AccName = fx_trades.AccName.str[0:3]
fx_trades["Contra Qty"].replace(to_replace=",", value="", regex=True, inplace=True)
fx_trades["Contra Qty"] = fx_trades["Contra Qty"].apply(pd.to_numeric)
fx_trades["Contra Qty"] = fx_trades["Contra Qty"].abs()
fx_trades["Dealt Amount"].replace(to_replace=",", value="", regex=True, inplace=True)
fx_trades["Dealt Amount"] = fx_trades["Dealt Amount"].apply(pd.to_numeric)
fx_trades["Dealt Amount"] = fx_trades["Dealt Amount"].abs()
fx_trades = fx_trades.rename({'Date':'date','Instrument':'instrument'},axis=1)
fx_trades["date"] = pd.to_datetime(fx_trades["date"])
fx_purge = fx_trades[fx_trades['ID'].apply(lambda x: len(x)>19)]
fx_trades = fx_trades.drop(fx_purge.index).reset_index()
fx_trades= fx_trades.drop(['ID','index'],axis=1)

In [149]:
fx_trades =fx_trades.rename(columns={'Dealt Amount':'Dealt Qty'})
trades = trades[['date', 'instrument',  'Dealt Qty']]
fx_trades = fx_trades[['date', 'instrument',  'Dealt Qty']]

cryptos =['BTC', 'BCH', 'EOS','ETH','LTC', 'XRP','LNK','UST']


def custom_instrument(x):
    if "/" in x:
        vals = x.split("/")
        if vals[0] in cryptos:
            return "CRYPTOS"
        else:
            return "CFDS"
trades["Type"] = trades["instrument"].apply(lambda x: custom_instrument(x))

fx_trades['Type']='FX'

gcx =  pd.concat([trades, fx_trades]).reset_index()
gcx= gcx.drop(['index'],axis=1)

gcx['Dealt Qty'] = gcx['Dealt Qty'].abs()

gcx = gcx.groupby(["date","instrument","Type"]).agg({"Dealt Qty":"sum"}).reset_index()

In [157]:
gcx

,date,instrument,Type,Dealt Qty
0,2021-12-01,AUD/CAD,FX,154000.00
1,2021-12-01,AUD/USD,FX,6418000.00
2,2021-12-01,BTC/USD,CRYPTOS,28.41
3,2021-12-01,CAD/JPY,FX,6000.00
4,2021-12-01,DJI/USD,CFDS,26.00
...,...,...,...,...
293,2021-12-15,NZD/JPY,FX,262000.00
294,2021-12-15,NZD/USD,FX,688000.00
295,2021-12-15,USD/CAD,FX,78000.00
296,2021-12-15,USD/CHF,FX,94000.00


In [155]:
## Importing Necessary Libraries
import requests
import json
import urllib3
import pandas as pd
from datetime import datetime
urllib3.disable_warnings()
import calendar
import warnings
import numpy as np
import os

## Getting Terminus Data Using ISPrime API
API_URL_BASE = 'https://api.isprimefx.com/api/'
USERNAME = 'operations@alphatrade.com.au'
PASSWORD = 'promptshadeimpose'



# login and extract the authentication token
response = requests.post(API_URL_BASE + 'login', 
headers={'Content-Type': 'application/json'},
data=json.dumps({'username': USERNAME, 'password': PASSWORD}), verify=False)

token = response.json()['token']
headers = {'X-Token': token}
#print(token)

 ## Data Function to get rid of Sat & Sun days in each week 
def findDay(date):
    dday = datetime.strptime(date, '%Y-%m-%d').weekday()
    return (calendar.day_name[dday])
  
## Getting SPOT Trades for required day/s 
data_1 = pd.DataFrame()
arr_1 = []

############## PLEASE CHANGE THE DATES HERE CAREFULLY
for x in range(1,16):  ### CHANGE THE RANGE
    params = {'date': '2021-12-'+ str(x), ### CHANGE THE DATE MONTH
    'tradeType':'Spot',
    'category':'FX'}
    
    y = '2021-12-'+str(int(x))   ### CHANGE THE DATE SIMILAR TO THE PARAMS
    if (findDay(y) == 'Saturday') or (findDay(y) == 'Sunday'):
        continue
    
    response = requests.get(API_URL_BASE + 'trades', headers=headers, params=params, verify=False)
    d = json.dumps(response.json(), indent=4)
    df = pd.read_json(d)
    arr_1.append(df)
   
for i in range(0, len(arr_1)):
    data_1 = pd.concat([data_1, arr_1[i]], ignore_index=True, sort=False)
    



############### PLEASE CHANGE THE DATES HERE CAREFULLY

df_1 = pd.DataFrame()
lis_1 = []
for x in range(1,16):   ### CHANGE THE RANGE
    params = {'date': '2021-12-'+ str(x), ### CHANGE THE DATE MONTH
    'tradeType':'Spot', 
    'category':'IndexSwap'}
    
    y = '2021-12-'+str(int(x))    ### CHANGE THE DATE SIMILAR TO THE PARAMS
    if (findDay(y) == 'Saturday') or (findDay(y) == 'Sunday'):
        continue
    
    response = requests.get(API_URL_BASE + 'trades', headers=headers, params=params, verify=False)
    d = json.dumps(response.json(), indent=4)
    df = pd.read_json(d)
    lis_1.append(df)
   
for i in range(0, len(lis_1)):
    df_1 = pd.concat([df_1, lis_1[i]], ignore_index=True, sort=False)

    
################ PLEASE CHANGE THE DATE HERE CAREFULLY
# other_1 = pd.DataFrame()
# marks_1 = []

# for x in range(1,16):  ### FOLLOW THE ABOVE RANGE
#     params = {'date': '2021-12-'+ str(x)} ### FOLLOW ABOVE
#     y = '2021-12-'+str(int(x))  ### FOLLOW ABOVE

#     if (findDay(y) == 'Saturday') or (findDay(y) == 'Sunday'):
#         continue
    
#     response = requests.get(API_URL_BASE + 'marks', headers=headers, params=params, verify=False)
#     e = json.dumps(response.json(), indent=4)
#     df3 = pd.read_json(e)
#     marks_1.append(df3)

# for i in range(0, len(marks_1)):
#     other_1 = pd.concat([other_1, marks_1[i]], ignore_index=True, sort=False)
    
# other = other_1
# other = other.rename(columns={'price':'eodPrice'})
# other.date = pd.to_datetime(other.date)


## Merged Spot and IndexSwap

merged = pd.concat([data_1, df_1])
mother = merged[['tradeType', 'category', 'clientReference', 'accountNo', 'tradeDate', 'valueDate', 'tradeDttm', 'instrumentName', 'instrument', 'tradedCurrency', 'tradedQuantity', 'baseQuantity', 'contraQuantity','price', 'side', 'cancelled']]
mother.tradeDate = pd.to_datetime(mother.tradeDate)
'''Taking the absolute value of the Base value and Contract value '''

mother['Base Absolute Amount']= mother['tradedQuantity'].abs()
mother['Contra Absolute Amount']= mother['contraQuantity'].abs()

### Dictionary for getting CFD's origianl value using original multipliers
instrument_dict = { 'IDX':{
                            'US': 10,
                            'DE':10,
                            'JP':100,
                            'UK':10,
                            'AU':10,
                            'HK':10,
                            'HKD':10},
                    'SPT':{'CO':10}
                  }

#### Function for getting the Origianl Contract value 
def cust_contract(x):
    placeholder = 0
    placebo = 0
    if '.' in x[0]:
        cols = x[0].split('.')
        if cols[0].startswith('SPT'):
            placebo+= x[1] * x[2] * instrument_dict[cols[0]][cols[1]]
            return placebo
        else:
            placeholder += x[1] * (x[2]/ instrument_dict[cols[0]][cols[1]])
            return placeholder
    else:
        return x[3]
mother['Contract'] = mother[['instrument', 'price','Base Absolute Amount', 'Contra Absolute Amount']].apply(cust_contract, axis = 1) 

### CountryDict dictionary will be used to get the eod for the corresponding instruments
countryDict = {
        "IDX.AU": "AUD/USD",
        "IDX.EU": "EUR/USD",
        "IDX.FR": "EUR/USD",
        "IDX.DE": "EUR/USD",
        "IDX.ES": "EUR/USD",
        "IDX.HK": "HKD/USD",
        "IDX.JP": "JPY/USD",
        "IDX.UK": "GBP/USD",
        "IDX.US": "USD/USD",
        "CO": "USD/USD",
        "BTC": "USD/USD"
}

mother.rename(columns={'tradeDate':'date'}, inplace=True)



piv = mother.groupby(["date","instrument","category"]).agg({"Base Absolute Amount":"sum"}).reset_index()

piv = piv.rename(columns={'category':'Type', 'Base Absolute Amount':'Dealt Qty'})

all_ =  pd.concat([gcx, piv]).reset_index()
all_= all_.drop(['index'],axis=1)
_all = all_.groupby(["date","instrument","Type"]).agg({"Dealt Qty":"sum"}).reset_index()

D:\Anaconda\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
<ipython-input-155-5712ef3d8fac>:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mother['Base Absolute Amount']= mother['tradedQuantity'].abs()
<ipython-input-155-5712ef3d8fac>:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [156]:
_all

,date,instrument,Type,Dealt Qty
0,2021-12-01,AUD/CAD,FX,554000.00
1,2021-12-01,AUD/USD,FX,8260000.00
2,2021-12-01,BTC/USD,CRYPTOS,28.41
3,2021-12-01,CAD/JPY,FX,6000.00
4,2021-12-01,DJI/USD,CFDS,26.00
...,...,...,...,...
320,2021-12-15,USD/CAD,FX,88000.00
321,2021-12-15,USD/CHF,FX,94000.00
322,2021-12-15,USD/JPY,FX,1158000.00
323,2021-12-15,XAG/USD,FX,1000.00


In [158]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

## Getting the eods
eod = pd.read_csv("C:\\Users\\oazed\\Downloads\\2021-12-15 215835 - Hub Symbol EOD Price History - default.csv")
eod = eod[['Timestamp','Symbol', 'Bid Price']]
eod = eod.rename(columns={'Timestamp':'date'})
eod["date"] = pd.to_datetime(eod["date"])
eod['date'] = eod['date'].dt.date
eod = eod.rename({'Symbol':'instrument'},axis=1)
### EOD dictionary for translate
eods_dict= {
    'US500' : 'SPM/USD',
    'NAS100': 'NDM/USD',
    'US30' : 'DJI/USD',
    'AUS200': 'AXI/AUD',
    'EUSTX50':'STX/EUR',
    'HK50': 'HSI/HKD',
    'JPN225': 'JPU/JPY',
    'UK100' : 'UKI/GBP',
    'GER30': 'GEM/EUR',
    'CN50' : 'CNI/USD',
    'FRA40':'FRI/EUR',
}
 
#### Function that translate
def abar_banao(x):
    ins = x['instrument']
    if ins in eods_dict.keys():
        return eods_dict[ins]
    else:
        return ins
    
vals = eod.apply(abar_banao, axis=1)
eod['instrument'] = vals

df = pd.read_csv('C:\\Users\\oazed\\Downloads\\2021-12-15 215701 - Hub Trades List - default.csv')
df = df[df['Filled Volume'].apply(lambda x: x != 0)].reset_index()

p1 = df[df['Request Source'].apply(lambda x: x  == 'Swap')]
df = df.drop(p1.index)
p2 = df[df['Request Source'].apply(lambda x: x  == 'Adjustment')]
df = df.drop(p2.index)
# DST and EST timezone adjustments
def timezone_adjustments(date1):
    date1 = date1.split('.', 1) # Remove miliseconds
    date1 = date1[0]
    date1 = datetime.strptime(date1, '%Y-%m-%d %H:%M:%S')
    if datetime(2021, 11, 7, 2, 0, 0) <= date1 <= datetime(2022, 3, 14, 2, 0, 0):
        date2 = date1 + timedelta(hours = 2)
    else:
        date2 = date1 + timedelta(hours = 3)    
    return date2
df['Executed Time'] = df['Executed Time'].apply(lambda x: timezone_adjustments(x))
df = df.rename(columns={'Core Symbol':'instrument', 'Executed Time':'date'})

df = df[['date','instrument','Request Source','Taker Name','Filled Volume','Maker Stream Name', 'Maker Executed Price','Volume in USD','Term CY Conv Ratio' ]]

cryptos =['BTC/USD', 'BCH/USD', 'EOS/USD','ETH/USD','LTC/USD', 'XRP/USD','LNK/USD','UST/USD']
cfds = [ 'US500','NAS100','US30' ,'AUS200','EUSTX50','HK50','JPN225','UK100' ,'GER30','CN50' ,'FRA40']
metals = ['XAU/USD','XAG/USD']
def custom_instrument_oz(x):
    vals = x['instrument']
    lis = ["CRYPTOS", "CFDS","FX","STOCKS", "METAL"]
    if vals in cryptos:
        return lis[0]
    if vals in cfds:
        return lis[1]
    if vals in metals:
        return lis[4]
    if "." in vals:
        return lis[3]
    else:
        return lis[2]
df["category"] = df.apply(custom_instrument_oz,axis=1)

def abar_banao(x):
    ins = x['instrument']
    if ins in eods_dict.keys():
        return eods_dict[ins]
    else:
        return ins
    
df_vals = df.apply(abar_banao, axis=1)
df['instrument'] = df_vals


other_dict= {
    'SPM/USD' : 'SPM/USD',
    'NDM/USD': 'NDM/USD',
    'DJI/USD' : 'DJI/USD',
    'AXI/AUD': 'AXI/AUD',
    'STX/EUR':'STX/EUR',
    'HSI/HKD': 'HSI/HKD',
    'JPU/JPY': 'JPU/JPY',
    'UKI/GB' : 'UKI/GBP',
    'GEM/EUR': 'GEM/EUR',
    'CNI/USD' : 'CNI/USD',
    'FRI/EUR':'FRI/EUR',
    'XBRUSD':'XBRUSD',
    'XTIUSD':'XTIUSD',
    'WTIUSD':'WTIUSD'
}

#### Function to add eod start here 
conv = dict()

def findUSD(s):
    date = s['date']
    ins = s['instrument']
    rate = s['Bid Price']
    
    if '/' not in ins:
        if ins.endswith('USD'):
            conv[date][ins] = rate
    x = ins.split('/')

    if date not in conv:
        conv[date] = dict()
        conv[date]['USD/USD'] = 1.0
    
    if len(x) != 2:
        return
        
    if x[1] == 'USD':
        conv[date][ins] = rate
        
    if x[0] == 'USD':
        ins = x[1] + '/' + x[0]
        conv[date][ins] = 1/rate

def applyUSD(s):
    date = s['date']
    ins = s['instrument']
    rate = s['Bid Price']
    
    x = ins.split('/')
    
    if len(x) != 2:
        for kwd in other_dict.keys():
            if kwd in ins:
                if other_dict[kwd] in conv[date]:
                    return conv[date][other_dict[kwd]]
     
    if 'USD' not in x:
        if x[0] == 'AXI':
            return conv[date]['AUD/USD'] * rate
        if x[0] == 'UKI':
            return conv[date]['GBP/USD'] * rate
        if x[0] == 'GEM':
            return  conv[date]['EUR/USD'] * rate
        if x[0] == 'HSI':
            return conv[date]['HKD/USD'] * rate
        if x[0] == 'JPU':
            return conv[date]['JPY/USD'] * rate
        if x[0] == 'FRI':
            return conv[date]['EUR/USD'] * rate
        
        
    if 'USD' in x:
        if x[0] == 'USD':
            return 1/rate
        elif x[1] == 'USD':
            return rate
    
    ins0 = x[0]+'/'+'USD'
#     ins1 = x[1]+'/'+'USD'
    
    if ins0 in conv[date]:
        return conv[date][ins0]
    
#     if ins1 in conv[date]:
#         return rate * conv[date][ins1]
        
eod.apply(findUSD, axis=1)

eod['EoD_IN_USD'] = eod.apply(applyUSD, axis=1)

eod.set_index(['date','instrument'], inplace=True)
df['date'] = df['date'].dt.date
df.set_index(['date','instrument'], inplace=True)

omni = df.join(eod, how='left')
omni= omni.reset_index()
omni['Vols_in_USD'] = omni['Filled Volume']* omni['EoD_IN_USD']

#### Handling USD trades 
def eod_usd(x):
    val1 = x['Vols_in_USD']
    val2 = x['Filled Volume']
    val3 = x['instrument']
    ins = val3.split('/')
    if ins[0] == 'USD':
        return val2
    else:
        return val1
    
omni['Vols_in_USD'] = omni.apply(eod_usd,axis=1)

#### ALL INSTRUMENT DUMP
# omni.to_excel(writer, sheet_name='Sheet1', index=False)

omni = omni.groupby(["date","instrument","category"]).agg({"Filled Volume":"sum"}).reset_index()
result = pd.concat([omni, _all], axis=1)
result.to_csv('C:\\Rebate\\202112 1-15 Reconcilliation.csv',index=False)